In [12]:
# Cell 1: Setup and Imports
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# Add project root to Python path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Load environment variables
load_dotenv(project_root / '.env')

# Test API key
assemblyai_key = os.getenv('ASSEMBLYAI_API_KEY')
print(f"AssemblyAI API Key loaded: {'✅' if assemblyai_key else '❌'}")
print(f"Key starts with: {assemblyai_key[:10] if assemblyai_key else 'None'}...")


AssemblyAI API Key loaded: ✅
Key starts with: 972365f41d...


In [17]:
# Cell 2: Import Dependencies
import assemblyai as aai
from langgraph.graph import StateGraph
from typing import TypedDict, Optional, List  # ← Added List here
import glob
import time
from pathlib import Path  # ← Also added Path here

# Import our database
from database.db_operations import db
from database.models import ConversationCreate

print("✅ All imports successful")

✅ All imports successful


In [ ]:
# Cell 3: Define LangGraph State
class AudioPipelineState(TypedDict):
    file_path: str
    filename: str
    transcript_text: Optional[str]
    conversation_id: Optional[int]
    extracted_insights: Optional[ExtractedInsights]  # ← NEW: Using our Pydantic model
    error: Optional[str]
    status: str

print("✅ State defined")

✅ State defined


In [15]:
# Cell 4: Test AssemblyAI Connection
# Configure AssemblyAI
aai.settings.api_key = os.getenv('ASSEMBLYAI_API_KEY')

# Test with a simple transcription (we'll use a file from temp folder)
def test_assemblyai_connection():
    """Test if AssemblyAI is working"""
    try:
        # Just test the API key is valid
        transcriber = aai.Transcriber()
        print("✅ AssemblyAI connection successful")
        return True
    except Exception as e:
        print(f"❌ AssemblyAI connection failed: {e}")
        return False

test_assemblyai_connection()

✅ AssemblyAI connection successful


True

In [18]:
# Cell 5: Batch File Discovery and Management
def find_audio_files(temp_folder: Path) -> List[Path]:
    """Find all audio files in temp folder"""
    audio_extensions = ['*.wav', '*.mp3', '*.m4a']
    audio_files = []
    
    for ext in audio_extensions:
        audio_files.extend(temp_folder.glob(ext))
    
    return sorted(audio_files)

def display_batch_info(audio_files: List[Path]):
    """Display information about the batch of files"""
    if not audio_files:
        print("❌ No audio files found in temp folder!")
        return False
    
    total_size_mb = sum(f.stat().st_size for f in audio_files) / (1024 * 1024)
    
    print(f"📊 BATCH PROCESSING INFO:")
    print(f"   Files to process: {len(audio_files)}")
    print(f"   Total size: {total_size_mb:.1f} MB")
    print(f"\n📁 Files found:")
    
    for i, file_path in enumerate(audio_files, 1):
        size_mb = file_path.stat().st_size / (1024 * 1024)
        print(f"   {i}. {file_path.name} ({size_mb:.1f} MB)")
    
    return True

def cleanup_processed_files(processed_files: List[Path]):
    """Delete all successfully processed files"""
    print(f"\n🗑️ CLEANUP: Deleting {len(processed_files)} processed files...")
    deleted_count = 0
    
    for file_path in processed_files:
        try:
            file_path.unlink()  # Delete file
            print(f"   ✅ Deleted: {file_path.name}")
            deleted_count += 1
        except Exception as e:
            print(f"   ❌ Failed to delete {file_path.name}: {e}")
    
    print(f"🗑️ Cleanup complete: {deleted_count}/{len(processed_files)} files deleted")

# Discover files in temp folder
temp_folder = project_root / 'data' / 'temp'
temp_folder.mkdir(parents=True, exist_ok=True)  # Ensure folder exists

audio_files = find_audio_files(temp_folder)
files_available = display_batch_info(audio_files)

if files_available:
    print(f"\n🚀 Ready to process {len(audio_files)} files!")
else:
    print("\n💡 TIP: Add .wav files to data/temp/ folder for testing")

📊 BATCH PROCESSING INFO:
   Files to process: 3
   Total size: 2.1 MB

📁 Files found:
   1. blog_barchthreee.wav (0.7 MB)
   2. blog_batchone.wav (0.9 MB)
   3. blog_batxhtwo.wav (0.5 MB)

🚀 Ready to process 3 files!


In [19]:
# Cell 6: Define LangGraph Nodes
def transcription_node(state: AudioPipelineState) -> AudioPipelineState:
    """Node 1: Transcribe audio file with AssemblyAI"""
    try:
        print(f"🎙️ Transcribing: {state['filename']}")
        
        # Configure transcriber
        transcriber = aai.Transcriber()
        
        # Transcribe the file
        transcript = transcriber.transcribe(state['file_path'])
        
        if transcript.status == aai.TranscriptStatus.error:
            return {
                **state,
                "error": f"AssemblyAI error: {transcript.error}",
                "status": "transcription_failed"
            }
        
        return {
            **state,
            "transcript_text": transcript.text,
            "status": "transcribed"
        }
        
    except Exception as e:
        return {
            **state,
            "error": f"Transcription error: {str(e)}",
            "status": "transcription_failed"
        }

def database_saver_node(state: AudioPipelineState) -> AudioPipelineState:
    """Node 2: Save transcript to database"""
    try:
        print(f"💾 Saving to database: {state['filename']}")
        
        # Create conversation object
        conversation = ConversationCreate(
            title=f"Audio: {state['filename']}",
            raw_text=state['transcript_text'],
            source="transcribed"
        )
        
        # Save to database
        conversation_id = db.create_conversation(conversation)
        
        return {
            **state,
            "conversation_id": conversation_id,
            "status": "completed"
        }
        
    except Exception as e:
        return {
            **state,
            "error": f"Database error: {str(e)}",
            "status": "database_failed"
        }

print("✅ LangGraph nodes defined")

✅ LangGraph nodes defined


In [20]:
# Cell: Initialize Database
from database.init_db import create_database

print("🔧 Checking/creating database...")
try:
    create_database()
    print("✅ Database ready!")
except Exception as e:
    print(f"❌ Database error: {e}")

# Verify tables exist
import sqlite3
conn = sqlite3.connect('data/app.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(f"📊 Available tables: {[table[0] for table in tables]}")
conn.close()

🔧 Checking/creating database...
Creating database at: data/app.db
✅ Database schema created successfully!
✅ Created tables: ['conversations', 'sqlite_sequence', 'blog_post_ideas', 'processing_status']
✅ Database ready!
📊 Available tables: ['conversations', 'sqlite_sequence', 'blog_post_ideas', 'processing_status']


In [21]:
# Cell 7: Build LangGraph Workflow
def build_pipeline():
    """Build the LangGraph workflow"""
    workflow = StateGraph(AudioPipelineState)
    
    # Add nodes
    workflow.add_node("transcribe", transcription_node)
    workflow.add_node("save_to_db", database_saver_node)
    
    # Add edges
    workflow.add_edge("transcribe", "save_to_db")
    workflow.set_entry_point("transcribe")
    workflow.set_finish_point("save_to_db")
    
    return workflow.compile()

# Build the pipeline
pipeline = build_pipeline()
print("✅ LangGraph pipeline compiled and ready")

✅ LangGraph pipeline compiled and ready


In [22]:
# Cell 8: Batch Processing Function
def process_audio_batch(audio_files: List[Path], pipeline) -> dict:
    """Process all audio files in batch"""
    
    if not audio_files:
        print("❌ No files to process")
        return {"processed": [], "failed": [], "total": 0}
    
    print(f"\n🚀 STARTING BATCH PROCESSING - {len(audio_files)} files")
    print("=" * 60)
    
    processed_files = []
    failed_files = []
    results = []
    
    for i, file_path in enumerate(audio_files, 1):
        print(f"\n📂 Processing {i}/{len(audio_files)}: {file_path.name}")
        print("-" * 40)
        
        # Create initial state
        initial_state = {
            "file_path": str(file_path),
            "filename": file_path.name,
            "transcript_text": None,
            "conversation_id": None,
            "error": None,
            "status": "processing"
        }
        
        try:
            # Run through pipeline
            result = pipeline.invoke(initial_state)
            
            if result["status"] == "completed":
                print(f"✅ SUCCESS: {file_path.name}")
                print(f"   Conversation ID: {result['conversation_id']}")
                print(f"   Transcript preview: {result['transcript_text'][:100]}...")
                processed_files.append(file_path)
            else:
                print(f"❌ FAILED: {file_path.name}")
                print(f"   Error: {result.get('error', 'Unknown error')}")
                failed_files.append(file_path)
            
            results.append(result)
            
        except Exception as e:
            print(f"❌ PIPELINE ERROR: {file_path.name}")
            print(f"   Exception: {str(e)}")
            failed_files.append(file_path)
            
            results.append({
                **initial_state,
                "error": str(e),
                "status": "pipeline_error"
            })
    
    return {
        "processed": processed_files,
        "failed": failed_files,
        "total": len(audio_files),
        "results": results
    }

print("✅ Batch processing function ready")

✅ Batch processing function ready


In [23]:
# Cell 9: Execute Batch Processing with Cleanup
if files_available:
    print("🎯 Starting batch processing...")
    
    # Process all files
    batch_results = process_audio_batch(audio_files, pipeline)
    
    # Display summary
    print(f"\n📊 BATCH PROCESSING COMPLETE!")
    print("=" * 60)
    print(f"✅ Successfully processed: {len(batch_results['processed'])}")
    print(f"❌ Failed: {len(batch_results['failed'])}")
    print(f"📁 Total files: {batch_results['total']}")
    
    # Show failed files
    if batch_results['failed']:
        print(f"\n❌ Failed files:")
        for file_path in batch_results['failed']:
            print(f"   - {file_path.name}")
    
    # Cleanup successfully processed files
    if batch_results['processed']:
        confirm = input(f"\n🗑️ Delete {len(batch_results['processed'])} processed files? (y/N): ")
        if confirm.lower() in ['y', 'yes']:
            cleanup_processed_files(batch_results['processed'])
        else:
            print("🔧 Files kept in temp folder for inspection")
    
    print("\n🎉 Batch processing complete!")
    
else:
    print("💡 Add audio files to data/temp/ folder and rerun this cell")

🎯 Starting batch processing...

🚀 STARTING BATCH PROCESSING - 3 files

📂 Processing 1/3: blog_barchthreee.wav
----------------------------------------
🎙️ Transcribing: blog_barchthreee.wav
💾 Saving to database: blog_barchthreee.wav
✅ SUCCESS: blog_barchthreee.wav
   Conversation ID: 2
   Transcript preview: Testing batch processing number three....

📂 Processing 2/3: blog_batchone.wav
----------------------------------------
🎙️ Transcribing: blog_batchone.wav
💾 Saving to database: blog_batchone.wav
✅ SUCCESS: blog_batchone.wav
   Conversation ID: 3
   Transcript preview: Testing batch processing one....

📂 Processing 3/3: blog_batxhtwo.wav
----------------------------------------
🎙️ Transcribing: blog_batxhtwo.wav
💾 Saving to database: blog_batxhtwo.wav
✅ SUCCESS: blog_batxhtwo.wav
   Conversation ID: 4
   Transcript preview: Testing batch process into....

📊 BATCH PROCESSING COMPLETE!
✅ Successfully processed: 3
❌ Failed: 0
📁 Total files: 3

🗑️ CLEANUP: Deleting 3 processed files...
 

In [24]:
# Cell 10: View Results in Database
def show_recent_conversations(limit=10):
    """Display recent conversations from database"""
    conversations = db.get_all_conversations()
    
    if not conversations:
        print("📝 No conversations found in database")
        return
    
    print(f"📝 Recent Conversations (showing {min(limit, len(conversations))}):")
    print("-" * 60)
    
    for conv in conversations[:limit]:
        print(f"ID: {conv.id} | Title: {conv.title}")
        print(f"Source: {conv.source} | Words: {conv.word_count} | Status: {conv.status}")
        print(f"Created: {conv.created_at}")
        print(f"Preview: {conv.raw_text[:150]}...")
        print("-" * 60)

# Show results
show_recent_conversations()

📝 Recent Conversations (showing 4):
------------------------------------------------------------
ID: 4 | Title: Audio: blog_batxhtwo.wav
Source: transcribed | Words: 4 | Status: pending
Created: 2025-09-24 08:20:07
Preview: Testing batch process into....
------------------------------------------------------------
ID: 3 | Title: Audio: blog_batchone.wav
Source: transcribed | Words: 4 | Status: pending
Created: 2025-09-24 08:20:02
Preview: Testing batch processing one....
------------------------------------------------------------
ID: 2 | Title: Audio: blog_barchthreee.wav
Source: transcribed | Words: 5 | Status: pending
Created: 2025-09-24 08:19:57
Preview: Testing batch processing number three....
------------------------------------------------------------
ID: 1 | Title: Audio: blog_recordcomtines.wav
Source: transcribed | Words: 30 | Status: pending
Created: 2025-09-24 07:48:54
Preview: I'm uploading a file from my telephone, and the idea is that this file is going to be monitored 

In [ ]:

## 1. Pydantic Model for Structured Output

```python
from pydantic import BaseModel, Field
from typing import List, Optional
from enum import Enum

class SpeakerRole(str, Enum):
    """Possible speaker roles in the conversation"""
    USER = "user"
    CLIENT = "client"
    POTENTIAL_CLIENT = "potential_client"
    INTERVIEWER = "interviewer"
    TEAM_MEMBER = "team_member"
    OTHER = "other"

class Speaker(BaseModel):
    """Information about a person speaking in the conversation"""
    name: Optional[str] = Field(description="Name of the speaker if mentioned")
    role: SpeakerRole = Field(description="Role of the speaker in the conversation")
    company: Optional[str] = Field(description="Company they work for if mentioned")

class Challenge(BaseModel):
    """A challenge or problem mentioned in the conversation"""
    description: str = Field(description="Description of the challenge")
    impact: str = Field(description="How this challenge affects them")
    urgency: str = Field(description="Low, Medium, or High urgency")

class CurrentSolution(BaseModel):
    """How they currently solve their problems"""
    solution: str = Field(description="What they're currently doing")
    satisfaction_level: str = Field(description="How satisfied they are: Very Satisfied, Satisfied, Neutral, Unsatisfied, Very Unsatisfied")
    limitations: List[str] = Field(description="Limitations of current solution")

class Need(BaseModel):
    """A need identified using psychology frameworks like NVC"""
    need_category: str = Field(description="Category of need (e.g., autonomy, efficiency, security, connection)")
    description: str = Field(description="Specific need description")
    intensity: str = Field(description="Low, Medium, or High intensity")

class ExtractedInsights(BaseModel):
    """Complete structured output from conversation analysis"""
    
    # Speakers
    speakers: List[Speaker] = Field(description="People identified in the conversation")
    
    # What they care about
    core_values: List[str] = Field(description="What this person/company cares about most")
    priorities: List[str] = Field(description="Their current priorities and focus areas")
    
    # Challenges
    primary_challenges: List[Challenge] = Field(description="Main problems they're facing")
    secondary_challenges: List[Challenge] = Field(description="Secondary or related problems")
    
    # Current solutions
    current_solutions: List[CurrentSolution] = Field(description="How they solve problems today")
    
    # Needs analysis
    psychological_needs: List[Need] = Field(description="Underlying needs using NVC or similar frameworks")
    
    # Keywords and themes
    problem_keywords: List[str] = Field(description="Key words/phrases they use to describe problems")
    main_themes: List[str] = Field(description="Overarching themes in the conversation")
    
    # Context
    conversation_summary: str = Field(description="Brief summary of the conversation")
    automation_context: str = Field(description="Specific context about their automation/tech challenges")
```

In [ ]:
## 3. PainExtractor Node Implementation

```python
import openai
import json
from typing import Dict, Any

# System prompt
PAIN_EXTRACTOR_SYSTEM_PROMPT = """
You are a UX researcher and business analyst for BigKids Automation. Your job is listening to transcripts from interviews with users and potential clients. 

You pay special attention to problems that users have regarding how their company is automating, using web apps and AI to save time and move towards a more ethical and sovereign tech infrastructure.

You will be given the transcript of an interview with a user or potential client.

Your task is to extract structured information about:
- Who is speaking and their role
- What this person cares about (values, priorities)
- Their main primary and secondary challenges
- How they are solving problems today
- Their underlying psychological needs (using frameworks like NVC - Non-Violent Communication)
- Keywords they use to describe their main problems

Focus on automation, web apps, AI, time-saving, ethical tech, and sovereign infrastructure themes.

Be thorough but concise. If information is not clearly stated, mark as null/empty rather than guessing.
"""

In [ ]:
def extract_insights_from_transcript(transcript_text: str, model="gpt-4o-2024-08-06", temperature=0) -> ExtractedInsights:
    """
    Extract structured insights from conversation transcript using OpenAI structured output
    """
    try:
        response = client.beta.chat.completions.parse(
            model=model,
            temperature=temperature,
            messages=[
                {"role": "system", "content": PAIN_EXTRACTOR_SYSTEM_PROMPT},
                {"role": "user", "content": transcript_text},
            ],
            response_format=ExtractedInsights
        )
        
        return response.choices[0].message.parsed
        
    except Exception as e:
        print(f"❌ Error extracting insights: {e}")
        return None

def pain_extractor_node(state: AudioPipelineState) -> AudioPipelineState:
    """
    LangGraph node: Extract structured insights from conversation transcript
    """
    print("🧠 Starting pain extraction...")
    
    try:
        # Extract insights using OpenAI structured output
        insights = extract_insights_from_transcript(state['transcript_text'])
        
        if insights:
            print(f"✅ Extracted insights: {len(insights.primary_challenges)} primary challenges, {len(insights.speakers)} speakers")
            
            return {
                **state,
                "extracted_insights": insights,
                "status": "insights_extracted"
            }
        else:
            return {
                **state,
                "error": "Failed to extract insights from transcript",
                "status": "error"
            }
            
    except Exception as e:
        print(f"❌ Pain extraction failed: {e}")
        return {
            **state,
            "error": f"Pain extraction error: {str(e)}",
            "status": "error"
        }
```


In [ ]:
## 4. Usage in Jupyter Notebook

```python
# Cell: Test Pain Extractor Node
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Test with sample transcript
test_state = {
    "file_path": "data/temp/test.wav",
    "filename": "test.wav",
    "transcript_text": "Hi, I'm John from TechCorp. We're struggling with our manual processes. We spend hours every day on data entry and it's killing our productivity. We've tried some automation tools but they don't integrate well with our existing systems. What we really need is something that respects our data sovereignty and doesn't lock us into big tech platforms.",
    "conversation_id": 1,
    "extracted_insights": None,
    "blog_ideas": None,
    "error": None,
    "status": "transcribed"
}

# Run the pain extractor node
result_state = pain_extractor_node(test_state)

# Display results
if result_state.get('extracted_insights'):
    insights = result_state['extracted_insights']
    print("🎯 Extracted Insights:")
    print(f"Speakers: {[s.name for s in insights.speakers]}")
    print(f"Primary challenges: {len(insights.primary_challenges)}")
    print(f"Keywords: {insights.problem_keywords}")
```